In [130]:
from src.SqlAlquemyInsertHandler import SqlAlquemyInsertHandler
import pandas as pd

from dotenv import load_dotenv
load_dotenv()

True

In [131]:
all_countries = ['United States', 'Japan', 'United Kingdom', 'Canada',
                 'France', 'Switzerland', 'Germany', 'Australia',
                 'Netherlands', 'Sweden', 'Spain', 'Hong Kong',
                 'Italy', 'Singapore', 'Belgium', 'Norway', 'Israel',
                 'Ireland', 'New Zealand', 'Austria', 'Euro Zone',
                 'China', 'Taiwan', 'India', 'South Korea', 'Brazil',
                 'Saudi Arabia', 'South Africa', 'Mexico', 'Indonesia',
                 'Türkiye', 'Poland', 'Argentina', 'Russia']

In [132]:
def process_quaterly_indicator(indicator, alt_indicator):
    sql_handler = SqlAlquemyInsertHandler()
    df = sql_handler.read_indicator(indicator, alt_indicator)

    df_processed = df.loc[:, ['ReportDateTime', 'Country', 'Indicator']]

    prefixes = ['Spanish ', 'French ', 'German ', 'Austrian ',
                'Italian ', 'Belgian ', 'Dutch ', 'Irish ']
    for prefix in prefixes:
        df_processed['Indicator'] = df_processed['Indicator'].str.removeprefix(
            prefix)

    suffixes = [' (Q1)', ' (Q2)', ' (Q3)', ' (Q4)']
    for suffix in suffixes:
        df_processed['Indicator'] = df_processed['Indicator'].str.removesuffix(
            suffix)

    df_processed['Indicator'].replace(alt_indicator, indicator, inplace=True)

    periods = pd.PeriodIndex(df_processed['ReportDateTime'], freq='Q')-1
    df_processed['Period'] = periods.to_timestamp()

    df_processed['Value'] = df['Actual'].str.rstrip('%').astype(float)/100

    df_processed = df_processed[df_processed['Indicator'] == indicator]

    missing_countries = [x for x in all_countries
                         if x not in df_processed['Country'].unique()]
    print('Missing countries')
    print(missing_countries)

    print('Avg publish delay: ', (df_processed['ReportDateTime'] -
                                  df_processed['Period']).mean())

    return df_processed

In [133]:
def transform_to_df_countries(df_processed):
    period = pd.period_range('2012-01-01', '2023-07-01', freq='Q')
    period = period.to_timestamp()

    df_processed.index = df_processed['Period']
    df_countries = pd.DataFrame(index=period, columns=all_countries)
    for country in all_countries:
        values = df_processed[df_processed['Country'] == country]['Value']
        values = values[~values.index.duplicated(keep='first')]
        df_countries.loc[:, country] = values

    return df_countries

# GDP (QoQ)

In [134]:
df_processed = process_quaterly_indicator('GDP (QoQ)', 'GDP Annualized (QoQ)')
df_processed

2273 options to calculate
Missing countries
['Taiwan', 'India', 'Saudi Arabia', 'Türkiye', 'Argentina', 'Russia']
Avg publish delay:  141 days 22:20:13.282850780


,ReportDateTime,Country,Indicator,Period,Value
0,2012-01-11 06:10:00,Euro Zone,GDP (QoQ),2011-10-01,0.001
1,2012-01-25 04:30:00,United Kingdom,GDP (QoQ),2011-10-01,-0.002
2,2012-01-25 18:00:00,South Korea,GDP (QoQ),2011-10-01,0.004
3,2012-01-27 08:30:00,United States,GDP (QoQ),2011-10-01,0.028
4,2012-01-30 03:00:00,Spain,GDP (QoQ),2011-10-01,-0.003
...,...,...,...,...,...
2268,2023-11-14 03:30:00,Netherlands,GDP (QoQ),2023-07-01,-0.002
2269,2023-11-14 04:00:00,Poland,GDP (QoQ),2023-07-01,0.014
2270,2023-11-14 05:00:00,Euro Zone,GDP (QoQ),2023-07-01,-0.001
2271,2023-11-14 18:50:00,Japan,GDP (QoQ),2023-07-01,-0.005


In [135]:
transform_to_df_countries(df_processed).tail(10)

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Sweden,...,South Korea,Brazil,Saudi Arabia,South Africa,Mexico,Indonesia,Türkiye,Poland,Argentina,Russia
2021-04-01,0.065,0.003,0.048,-0.003,0.009,0.018,0.015,0.007,0.031,0.009,...,0.007,-0.001,NaN,0.012,0.015,0.0331,NaN,0.019,NaN,NaN
2021-07-01,0.020,-0.008,0.013,0.013,0.03,0.017,0.018,-0.019,0.019,0.018,...,0.003,-0.001,NaN,-0.015,-0.002,0.0155,NaN,0.021,NaN,NaN
2021-10-01,0.069,0.013,0.01,0.016,0.007,0.003,-0.007,0.034,0.009,0.014,...,0.011,0.005,NaN,NaN,-0.001,0.0106,NaN,0.017,NaN,NaN
2022-01-01,-0.014,-0.002,0.008,0.008,0.0,0.005,0.002,0.008,0.0,-0.004,...,0.007,0.01,NaN,0.019,0.009,-0.0096,NaN,0.024,NaN,NaN
2022-04-01,-0.009,0.005,0.004,0.008,0.005,0.003,0.0,0.009,0.026,0.014,...,0.007,0.012,NaN,-0.007,0.01,0.0372,NaN,-0.023,NaN,NaN
2022-07-01,0.026,-0.003,-0.002,0.007,0.002,0.002,0.003,0.006,-0.002,0.007,...,0.003,0.004,NaN,0.016,0.01,0.0181,NaN,0.009,NaN,NaN
2022-10-01,0.029,0.002,0.0,0.0,0.001,0.0,-0.002,0.005,0.006,-0.006,...,-0.004,-0.002,NaN,-0.013,0.004,0.0036,NaN,-0.024,NaN,NaN
2023-01-01,0.011,0.004,0.001,0.008,0.002,0.003,-0.001,0.002,-0.007,0.002,...,0.003,0.019,NaN,0.004,0.011,-0.0092,NaN,0.039,NaN,NaN
2023-04-01,0.024,0.015,0.002,0.0,0.005,0.0,0.0,0.004,-0.003,-0.015,...,0.006,0.009,NaN,0.006,0.009,0.0386,NaN,-0.037,NaN,NaN
2023-07-01,0.049,-0.005,0.0,NaN,0.001,NaN,-0.001,NaN,-0.002,0.0,...,0.006,NaN,NaN,NaN,0.009,0.016,NaN,0.014,NaN,NaN


# GDP (YoY)

In [136]:
df_processed = process_quaterly_indicator('GDP (YoY)', 'GDP Quarterly (YoY)')
df_processed

1923 options to calculate
Missing countries
['United States', 'Belgium', 'Norway', 'Israel', 'Austria']
Avg publish delay:  142 days 07:53:04.528301886


,ReportDateTime,Country,Indicator,Period,Value
0,2012-01-16 21:00:00,China,GDP (YoY),2011-10-01,0.089
1,2012-01-31 03:00:00,Taiwan,GDP (YoY),2011-10-01,0.019
2,2012-02-05 23:00:00,Indonesia,GDP (YoY),2011-10-01,0.066
3,2012-02-15 03:30:00,Netherlands,GDP (YoY),2011-10-01,-0.007
4,2012-02-16 00:00:00,Singapore,GDP (YoY),2011-10-01,0.036
...,...,...,...,...,...
1918,2023-11-14 03:30:00,Netherlands,GDP (YoY),2023-07-01,-0.006
1919,2023-11-14 04:00:00,Poland,GDP (YoY),2023-07-01,0.004
1920,2023-11-14 05:00:00,Euro Zone,GDP (YoY),2023-07-01,0.001
1921,2023-11-14 18:50:00,Japan,GDP (YoY),2023-07-01,-0.021


In [137]:
transform_to_df_countries(df_processed).tail(10)

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Sweden,...,South Korea,Brazil,Saudi Arabia,South Africa,Mexico,Indonesia,Türkiye,Poland,Argentina,Russia
2021-04-01,NaN,0.013,0.222,0.1272,0.187,0.077,0.096,0.096,0.097,0.1,...,0.059,0.124,0.018,0.193,0.197,0.0707,NaN,0.109,0.179,0.103
2021-07-01,NaN,-0.03,0.069,0.0397,0.033,0.041,0.025,0.039,0.05,0.047,...,0.04,0.04,0.07,0.029,0.046,0.0351,NaN,0.051,0.119,0.043
2021-10-01,NaN,0.054,0.065,0.038,0.054,0.037,0.014,0.042,0.062,0.062,...,0.041,0.016,0.068,0.017,0.01,0.0502,NaN,0.057,0.086,NaN
2022-01-01,NaN,-0.01,0.095,0.0289,0.053,0.044,0.04,0.033,0.07,0.03,...,0.031,0.017,0.099,0.03,0.016,0.0501,NaN,0.085,0.06,0.05
2022-04-01,NaN,0.022,0.035,0.0456,0.042,0.028,0.015,0.036,0.053,0.042,...,0.029,0.032,0.118,0.002,0.021,0.0544,NaN,0.053,0.069,-0.04
2022-07-01,NaN,-0.012,0.02,0.0393,0.01,0.005,0.011,0.059,0.031,0.026,...,0.031,0.036,0.086,0.041,0.042,0.0572,NaN,0.035,0.059,-0.04
2022-10-01,NaN,0.006,0.002,0.028,0.005,0.008,0.005,0.027,0.03,-0.006,...,0.014,0.019,0.055,0.009,0.036,0.0501,NaN,0.049,0.019,NaN
2023-01-01,NaN,0.016,0.002,0.0221,0.008,0.006,0.002,0.023,0.019,0.003,...,0.008,0.04,0.038,0.002,0.039,0.0503,NaN,-0.002,0.013,-0.027
2023-04-01,NaN,0.06,0.004,0.0112,0.009,0.005,-0.002,0.021,-0.003,-0.024,...,0.009,0.034,0.012,0.016,0.037,0.0517,NaN,-0.005,-0.049,0.049
2023-07-01,NaN,-0.021,0.006,NaN,0.007,NaN,-0.003,NaN,-0.006,-0.012,...,0.014,NaN,-0.045,NaN,0.033,0.0494,NaN,0.004,NaN,0.055


# Manufacturing PMI

In [138]:
def process_pmi_indicator(indicator, alt_indicator = ''):
    sql_handler = SqlAlquemyInsertHandler()
    df = sql_handler.read_indicator(indicator, alt_indicator)

    df_processed = df.loc[:, ['ReportDateTime', 'Country', 'Indicator']]

    prefixes = [
        'S&P Global Hong Kong ',
        'S&P Global Canada ',
        'S&P Global/CIPS UK ',
        'S&P Global / CIPS UK ',
        'S&P Global US ',
        'S&P Global Mexico ',
        'S&P Global India ',
        'S&P Global South Korea ',
        'HCOB Germany ',
        'HCOB Eurozone ',
        'S&P Global ',
        'Judo Bank Australia ',
        'au Jibun Bank Japan ',
        'HCOB France ',
        'S&P Global Taiwan ',
        'Unicredit Bank Austria ',
        'Nikkei ',
        'AIB Ireland ',
        'Caixin ',
        'Russian S&P Global ',
        'Poland ',
        'HCOB Spain ',
        'HCOB Italy ',
        'Taiwan ',
        'Riyad Bank Saudi Arabia ',
        'Chinese '
    ]
    for prefix in prefixes:
        df_processed['Indicator'] = df_processed['Indicator'].str.removeprefix(
            prefix)

    suffixes = [' (Jan)', ' (Feb)', ' (Mar)', ' (Apr)', ' (May)', ' (Jun)',
                ' (Jul)', ' (Aug)', ' (Sep)', ' (Oct)', ' (Nov)', ' (Dec)',
                ' (MoM)']
    for suffix in suffixes:
        df_processed['Indicator'] = df_processed['Indicator'].str.removesuffix(
            suffix)

    df_processed['Indicator'].replace(alt_indicator, indicator, inplace=True)

    periods = pd.PeriodIndex(df_processed['ReportDateTime'], freq='M')
    df_processed['Period'] = periods.to_timestamp()

    # correct period when logged at the beginning of the month
    prev_periods = pd.PeriodIndex(df_processed[
        df_processed['ReportDateTime'].dt.day < 15]['ReportDateTime'], freq='M')-1
    df_processed.loc[df_processed['ReportDateTime'].dt.day < 15, 'Period'] = prev_periods.to_timestamp()

    df_processed['Value'] = df['Actual'].astype(float)

    df_processed = df_processed[df_processed['Indicator'] == indicator]

    missing_countries = [x for x in all_countries
                        if x not in df_processed['Country'].unique()]
    print('Missing countries')
    print(missing_countries)

    print('Avg publish delay: ', (df_processed['ReportDateTime'] -
                                df_processed['Period']).mean())

    return df_processed

In [139]:
manufacturing_pmi = process_pmi_indicator('Manufacturing PMI', 'procure.ch PMI')
manufacturing_pmi.tail(28)

4914 options to calculate
Missing countries
['Netherlands', 'Singapore', 'Belgium', 'Israel', 'New Zealand', 'Saudi Arabia', 'Türkiye', 'Argentina']
Avg publish delay:  29 days 07:24:11.123595505


,ReportDateTime,Country,Indicator,Period,Value
4883,2023-10-27 03:00:00,Austria,Manufacturing PMI,2023-10-01,41.7
4884,2023-10-30 20:30:00,China,Manufacturing PMI,2023-10-01,49.5
4886,2023-10-31 17:00:00,Australia,Manufacturing PMI,2023-10-01,48.2
4887,2023-10-31 19:30:00,South Korea,Manufacturing PMI,2023-10-01,49.8
4888,2023-10-31 19:30:00,Japan,Manufacturing PMI,2023-10-01,48.7
4889,2023-10-31 19:30:00,Taiwan,Manufacturing PMI,2023-10-01,47.6
4890,2023-10-31 19:30:00,Indonesia,Manufacturing PMI,2023-10-01,51.5
4891,2023-10-31 20:01:00,Ireland,Manufacturing PMI,2023-10-01,48.2
4892,2023-10-31 20:45:00,China,Manufacturing PMI,2023-10-01,49.5
4893,2023-11-01 00:00:00,India,Manufacturing PMI,2023-10-01,55.5


In [140]:
services_pmi = process_pmi_indicator('Services PMI')
services_pmi.tail(15)

2635 options to calculate
Missing countries
['Canada', 'Switzerland', 'Netherlands', 'Hong Kong', 'Singapore', 'Belgium', 'Norway', 'Israel', 'New Zealand', 'Austria', 'Taiwan', 'South Korea', 'Saudi Arabia', 'South Africa', 'Mexico', 'Indonesia', 'Türkiye', 'Poland', 'Argentina']
Avg publish delay:  30 days 11:19:05.669829222


,ReportDateTime,Country,Indicator,Period,Value
2620,2023-11-02 17:00:00,Australia,Services PMI,2023-10-01,47.9
2621,2023-11-02 20:01:00,Ireland,Services PMI,2023-10-01,52.6
2622,2023-11-02 20:45:00,China,Services PMI,2023-10-01,50.4
2623,2023-11-03 00:00:00,India,Services PMI,2023-10-01,58.4
2624,2023-11-03 01:00:00,Russia,Services PMI,2023-10-01,53.6
2625,2023-11-03 02:30:00,Sweden,Services PMI,2023-10-01,48.5
2626,2023-11-03 04:30:00,United Kingdom,Services PMI,2023-10-01,49.5
2627,2023-11-03 08:45:00,United States,Services PMI,2023-10-01,50.6
2628,2023-11-05 19:30:00,Japan,Services PMI,2023-10-01,51.6
2629,2023-11-06 03:15:00,Spain,Services PMI,2023-10-01,51.1


In [141]:
composite_pmi = process_pmi_indicator('Composite PMI')
composite_pmi.tail(9)

1392 options to calculate
Missing countries
['Japan', 'Canada', 'Switzerland', 'Australia', 'Netherlands', 'Sweden', 'Spain', 'Hong Kong', 'Singapore', 'Belgium', 'Norway', 'Israel', 'Ireland', 'New Zealand', 'Austria', 'Taiwan', 'India', 'South Korea', 'South Africa', 'Mexico', 'Indonesia', 'Türkiye', 'Poland', 'Argentina', 'Russia']
Avg publish delay:  29 days 13:35:37.456896551


,ReportDateTime,Country,Indicator,Period,Value
1383,2023-10-30 20:30:00,China,Composite PMI,2023-10-01,50.7
1384,2023-11-03 04:30:00,United Kingdom,Composite PMI,2023-10-01,48.7
1385,2023-11-03 08:45:00,United States,Composite PMI,2023-10-01,50.7
1386,2023-11-04 23:15:00,Saudi Arabia,Composite PMI,2023-10-01,58.4
1387,2023-11-06 03:45:00,Italy,Composite PMI,2023-10-01,47.0
1388,2023-11-06 03:50:00,France,Composite PMI,2023-10-01,44.6
1389,2023-11-06 03:55:00,Germany,Composite PMI,2023-10-01,45.9
1390,2023-11-06 04:00:00,Euro Zone,Composite PMI,2023-10-01,46.5
1391,2023-11-06 08:00:00,Brazil,Composite PMI,2023-10-01,50.3


In [142]:
construction_pmi = process_pmi_indicator('Construction PMI')
construction_pmi.tail(5)

235 options to calculate
Missing countries
['United States', 'Japan', 'Canada', 'Switzerland', 'Australia', 'Netherlands', 'Sweden', 'Spain', 'Hong Kong', 'Singapore', 'Belgium', 'Norway', 'Israel', 'Ireland', 'New Zealand', 'Austria', 'China', 'Taiwan', 'India', 'South Korea', 'Brazil', 'Saudi Arabia', 'South Africa', 'Mexico', 'Indonesia', 'Türkiye', 'Poland', 'Argentina', 'Russia']
Avg publish delay:  34 days 02:11:21.702127659


,ReportDateTime,Country,Indicator,Period,Value
230,2023-11-06 04:30:00,United Kingdom,Construction PMI,2023-10-01,45.6
231,2023-11-07 03:30:00,Italy,Construction PMI,2023-10-01,51.8
232,2023-11-07 03:30:00,Germany,Construction PMI,2023-10-01,38.3
233,2023-11-07 03:30:00,France,Construction PMI,2023-10-01,41.0
234,2023-11-07 03:30:00,Euro Zone,Construction PMI,2023-10-01,42.7
